In [1]:
import geopandas as gpd
import pandas as pd
from shapely import Point, Polygon
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pyproj
from osgeo import gdal
import numpy as np
import rioxarray as rxr
from shapely import box
import whitebox
import rasterio as rio
from rasterio.plot import show,show_hist
wbt = whitebox.WhiteboxTools()
from osgeo import gdal

In [2]:
gdf_tx_aoi = gpd.read_file(r"datasets/tx_aoi.shp")

In [4]:
gdf_tx_aoi = gdf_tx_aoi.to_crs(3857)

In [12]:
gdf_tx_aoi.head()

case_id    faa_ors            faa_asn  usgs_pr_id  eia_id t_state  \
0  3119713  48-186736   2019-WTW-9999-OE         NaN     NaN      TX   
1  3118880  48-186804  2019-WTW-10122-OE         NaN     NaN      TX   
2  3120174  48-186775  2019-WTW-10009-OE         NaN     NaN      TX   
3  3119010  48-186800  2019-WTW-10079-OE         NaN     NaN      TX   
4  3120170  48-186801  2019-WTW-10083-OE         NaN     NaN      TX   

     t_county t_fips           p_name p_year  ... t_ttlh retrofit retrofit_y  \
0  Bee County  48025  Blackjack Creek   2022  ...  185.5        0        NaN   
1  Bee County  48025  Blackjack Creek   2022  ...  185.5        0        NaN   
2  Bee County  48025  Blackjack Creek   2022  ...  185.5        0        NaN   
3  Bee County  48025  Blackjack Creek   2022  ...  185.5        0        NaN   
4  Bee County  48025  Blackjack Creek   2022  ...  185.5        0        NaN   

  t_conf_atr t_conf_loc t_img_date     t_img_srce       xlong       ylat  \
0          1          3  2/17/2022  Digital Globe  -97.565788  28.285681   
1          1          3  3/22/2022  Digital Globe  -97.514153  28.240862   
2          1          3  2/17/2022  Digital Globe  -97.543541  28.293280   
3          1          3  3/22/2022  Digital Globe  -97.544586  28.265989   
4          1          3  2/17/2022  Digital Globe  -97.512680  28.283510   

                            geometry  
0  POINT (-10860973.839 3285039.606)  
1  POINT (-10855225.857 3279375.056)  
2  POINT (-10858497.314 3286000.259)  
3  POINT (-10858613.643 3282550.495)  
4  POINT (-10855061.883 3284765.165)  

[5 rows x 28 columns]

In [10]:
def reproject_raster(input_fpath, out_fpath, crs):
    """Function to reproject raster"""
    # Open the raster using rioxarray
    raster = rxr.open_rasterio(input_fpath)
    # Reproject the raster
    reprojected_raster = raster.rio.reproject(crs)
    # Save the reprojected raster to a new file
    reprojected_raster.rio.to_raster(out_fpath)
    return out_fpath

In [11]:
reproject_raster('datasets/USGS_13_n29w098_20211103.tif', 'datasets/DEM_aoi.tif', 3857)

In [ ]:
def create_cummulative_viewshed():
    # Whitebox viewshed uses a clip DEM raster, TX AoI Wind Turbine shapefile and creates a "Viewshed Raster" as an output
    wbt.viewshed('clip_raster.tif', 'tx_aoi.shp', 'viewshed_out.tif', height=2.0)